In [ ]:
import os

filename="outputs/ComfyUI_00309_.png"

file=os.path.basename(filename)
file

In [ ]:
from llama_cpp import Llama
import json

model_id="/Users/janghyeonbin/ai-companion/models/llm/gguf/Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"
model=Llama(model_id)

chat="안녕?"
response = model.create_chat_completion(
    messages=[
        {"role": "system", "content": "당신은 유용한 AI 비서입니다."},
        {"role": "user", "content": chat}
    ]
)

response["choices"][0]["message"]["content"]

In [ ]:
from diffusers import AutoPipelineForText2Image


In [ ]:
pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5")

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
snapshot_download(repo_id="faridlazuarda/valadapt-llama-3.1-8B-it-korean", local_dir="./models/llm/loras/valadapt-llama-3.1-8B-it-korean")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path="/Volumes/EXDATA/models/LLM/transformers/llama/meta-llama__Llama-3.1-8B-Instruct"
lora_path="/Users/janghyeonbin/ai-companion/models/llm/loras/valadapt-llama-3.1-8B-it-korean"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")
model.load_adapter(lora_path)

In [ ]:
import cv2
import imutils
import pytesseract

In [ ]:
image=cv2.imread('./outputs/ComfyUI_00318_.png')

In [ ]:
image.shape

In [ ]:
from PIL import Image, ImageOps

In [ ]:
original_image=Image.open('ComfyUI_00318_.png').convert('RGBA')
original_image.show()

In [ ]:
image=Image.open('layer_0.png').convert('RGBA')
image.show()

In [ ]:
new_alpha=image.getchannel('A')
new_alpha.show()

In [ ]:
new_alpha=ImageOps.invert(new_alpha)
new_alpha.show()

In [ ]:
new_mask=Image.new("RGBA", image.size)
new_mask.show()

In [ ]:
new_mask.putalpha(new_alpha)
new_mask.show()

In [ ]:
from huggingface_hub import HfApi

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
import os

In [ ]:
token=os.getenv('HF_TOKEN')
token

In [ ]:
api=HfApi(token=token)

In [ ]:
models=api.list_models()

In [ ]:
models=api.list_models(task='audio-text-to-text', library="transformers")

In [ ]:
list(models)

In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import matplotlib.pyplot as plt
import scipy.fftpack
import scipy.io.wavfile
import numpy as np

In [ ]:
def text2mp3():
    tts=gTTS('Hello',lang='en')
    tts.save('text1.mp3')

In [ ]:
text2mp3()

In [ ]:
def text2wav():
    tts=gTTS('Hello', lang="en")
    tts.save('hello.mp3')
    
    w=AudioSegment.from_mp3('hello.mp3')
    w.export('Hello.wav', format='wav')

In [ ]:
text2wav()

In [ ]:
def text2wav(text, lang):
    tts=gTTS(text, lang=lang)
    tts.save(f"{text}.mp3")
    
    w=AudioSegment.from_mp3(f"{text}.mp3")
    w.export(f"{text}.wav", format="wav")

In [ ]:
text2wav('Hello', lang='en')

In [ ]:
text2wav('안녕하세요', lang='ko')

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(path='characters_info', glob="*.txt", loader_cls=TextLoader)
docs = loader.load()
docs.sort(key=lambda x: x.metadata.get('source', ''))

len(docs)

In [ ]:
docs

In [ ]:
import os
import re
from langchain_classic.docstore.document import Document

In [ ]:
def split_document_by_sections(text):
    """
    텍스트에서 [항목] 형태의 섹션을 찾아서 (섹션 제목, 내용) 튜플 리스트로 반환하는 함수.
    """
    # 정규표현식 패턴: 대괄호 안에 항목명, 그 다음 내용은 다음 대괄호가 나오기 전까지
    pattern = re.compile(r'\[(.*?)\]\s*(.*?)(?=\n\s*\[|$)', re.DOTALL)
    sections = []
    for match in pattern.finditer(text):
        section_title = match.group(1).strip()
        section_content = match.group(2).strip()
        sections.append((section_title, section_content))
    return sections

In [ ]:
# 언어 이름과 코드 매핑 딕셔너리
LANGUAGE_MAP = {
    "한국어": "ko",
    "日本語": "ja",
    "简体中文": "zh_CN",
    "繁體中文": "zh_TW",
    "English": "en"
}

def extract_language_code(heading):
    """
    헤딩 텍스트 (예: "## 한국어")에서 언어 이름을 추출하여
    매핑 딕셔너리를 통해 언어 코드를 반환하는 함수.
    """
    match = re.match(r"##\s*(.+)", heading)
    if match:
        lang_name = match.group(1).strip()
        return LANGUAGE_MAP.get(lang_name, lang_name)
    return None


In [ ]:
def process_multilang_document(doc):
    """
    파일 하나에 여러 언어 섹션이 있을 때,
    각 언어 헤딩(예: "## 한국어")을 기준으로 블록을 분리하고,
    각 블록 내에서 [설정], [성격] 등 섹션별로 분리한 Document 리스트를 반환.
    """
    text = doc.page_content
    # 언어 헤딩(## 로 시작하는 줄)을 기준으로 분리 (헤딩을 포함하도록 분리)
    blocks = re.split(r'(?=##\s*)', text)
    
    processed = []
    for block in blocks:
        if block.startswith("##"):
            # 첫 줄에서 언어 헤딩을 추출
            lines = block.splitlines()
            lang_code = extract_language_code(lines[0])
            # 헤딩 부분을 제거한 나머지 내용 사용
            content = "\n".join(lines[1:]).strip()
            # 이전에 정의한 섹션 분리 함수 사용
            sections = split_document_by_sections(content)
            for section_title, section_content in sections:
                new_doc = Document(
                    page_content=section_content,
                    metadata={
                        "language": lang_code,
                        "section": section_title,
                        "source": doc.metadata.get("source", "unknown")
                    }
                )
                processed.append(new_doc)
        else:
            # 만약 블록이 언어 헤딩 없이 나온다면, 무시하거나 기본값 설정 가능
            pass
    return processed

In [ ]:
all_processed_docs = []

In [ ]:
for doc in docs:
    print(doc)
    processed_docs = process_multilang_document(doc)
    all_processed_docs.extend(processed_docs)


In [ ]:
len(all_processed_docs)

In [ ]:
for doc in all_processed_docs:
    print(doc.metadata, doc)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.vectorstores.utils import DistanceStrategy

In [ ]:
embedding_model_name = "BAAI/bge-m3"

In [ ]:
embedding_model=HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
vectorstore=FAISS.from_documents(
    documents=all_processed_docs, embedding=embedding_model, distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE
)

In [ ]:
vector_count = vectorstore.index.ntotal

In [ ]:
vector_count

In [ ]:
user_query = "미나미 아스카의 성격과 외모에 대해 어떻게 생각해?"

In [ ]:
query_vector = embedding_model.embed_query(user_query)

In [ ]:
len(query_vector)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
basic_docs = retriever.invoke(user_query)

In [ ]:
for doc in basic_docs:
    print(doc.metadata, doc)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')

In [ ]:
tokenizer.chat_template

In [ ]:
from transformers import AutoConfig

In [ ]:
config = AutoConfig.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')

In [ ]:
config.top_k

In [ ]:
config.top_p

In [ ]:
config.eos_token_id

In [ ]:
config.pad_token_id

In [ ]:
config.temperature

In [ ]:
config.repetition_penalty

In [ ]:
import gradio as gr

gr.themes.builder()

In [ ]:
import os
from pathlib import Path
import torch
from safetensors import safe_open
from safetensors.torch import save_file
from tqdm.auto import tqdm

# 変換するディレクトリを指定します
input_dir = "/tmp/ComfyUI/models/checkpoints/"

# 指定したディレクトリ内のすべてのsafetensorsファイルを取得
safetensors_files = list(Path(input_dir).glob("*.safetensors"))

def convert_model(file_path: str):
    print(f"変換中: {file_path}")
    
    # メタデータの読み込み
    try:
        with safe_open(file_path, framework="pt", device="cpu") as f:
            metadata = f.metadata()
            metadata = metadata if metadata is not None else {}
    except Exception as e:
        print(f"メタデータの読み取り中にエラーが発生しました: {str(e)}")
        return False

    # モデルの変換
    try:
        sd_pruned = {}
        with safe_open(file_path, framework="pt", device="cpu") as f:
            for key in tqdm(f.keys(), desc="テンソルを変換中"):
                tensor = f.get_tensor(key)
                sd_pruned[key] = tensor.to(torch.float8_e4m3fn)

        # 変換したモデルを保存（元のディレクトリに）
        output_dir = os.path.dirname(file_path)
        model_name = os.path.splitext(os.path.basename(file_path))[0]
        output_filename = f"{model_name}_fp8.safetensors"
        output_path = os.path.join(output_dir, output_filename)
        save_file(sd_pruned, output_path, metadata={"format": "pt", **metadata})
        print(f"ファイルが正常に保存されました: {output_path}")

        return True
    except Exception as e:
        print(f"変換中にエラーが発生しました: {str(e)}")
        return False
        

In [ ]:
file_path = "models/diffusion/checkpoints/sdxl/animagine-xl-4.0-opt.safetensors"

In [ ]:
convert_model(file_path)

In [ ]:
def convert_model_to_int8(file_path: str):
    print(f"Converting: {file_path}")
    
    # Read metadata
    try:
        with safe_open(file_path, framework="pt", device="cpu") as f:
            metadata = f.metadata()
            metadata = metadata if metadata is not None else {}
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return False

    # Convert the model
    try:
        sd_pruned = {}
        with safe_open(file_path, framework="pt", device="cpu") as f:
            for key in tqdm(f.keys(), desc="Converting tensors"):
                tensor = f.get_tensor(key)
                sd_pruned[key] = tensor.to(torch.float8_e4m3fn)

        # Save the converted model (in the original directory)
        output_dir = os.path.dirname(file_path)
        model_name = os.path.splitext(os.path.basename(file_path))[0]
        output_filename = f"{model_name}_fp8.safetensors"
        output_path = os.path.join(output_dir, output_filename)
        save_file(sd_pruned, output_path, metadata={"format": "pt", **metadata})
        print(f"Saved: {output_path}")

        return True
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return False
        

In [11]:
import os
import re
from datetime import datetime
from huggingface_hub import HfApi
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
token=os.getenv('HF_TOKEN')

In [ ]:
api = HfApi(token=token)

In [ ]:
api.create_repo(repo_id="mlx-community/aya-expanse-8b-8bit",token=token)
api.upload_folder(
    folder_path="models/llm/mlx/CohereForAI__aya-expanse-8b-8bit",
    repo_id="mlx-community/aya-expanse-8b-8bit",
    token=token,
)

In [ ]:
api.upload_folder(
    folder_path="models/diffusion/loras/sd35_medium/Tomboy_for_SD3.5_Medium_test",
    repo_id="bean980310/Tomboy_for_SD3.5_Medium_test",
    ignore_patterns=["._*"],
    token=token
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage

system_message = SystemMessage(content="당신은 유용한 AI 비서입니다.")
user_message = HumanMessage(content="안녕?")

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message.content),
    ("user", "{input}"),
])

In [ ]:
messages = prompt.format_messages(input=user_message.content)
messages

In [ ]:
from pathlib import Path
import openai

In [ ]:
openai.files.create(
    file=Path("/Users/janghyeonbin/Pictures/ComfyUI_00085_.png")
)

In [ ]:
from openai import OpenAI
import sys
import time

openai = OpenAI()

video = openai.videos.create(
    model="sora-2-pro",
    input_reference=Path("/Users/janghyeonbin/Pictures/ComfyUI_00085_resized.png"),
    seconds="8",
    size="1280x720",
    prompt="Minami Asuka the tomboyish girl is dual wielding swords. her short hair and clothes flutter in the wind. she says 「みんな、俺についてきて！」",
)

print(video.id)

In [ ]:
page = openai.videos.list()

In [ ]:
page = page.data[0]

In [ ]:
page.id

In [ ]:
response = openai.videos.download_content(
    video_id=video.id
)

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI()

In [14]:
model = client.models.list()

In [15]:
model

SyncPage[Model](data=[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-5.1-codex-mini', created=1763007109, object='model', owned_by='system'), Model(id='gpt-5.1-chat-latest', created=1762547951, object='model', owned_by='system'), Model(id='gpt-5.1-2025-11-13', created=1762800353, object='model', owned_by='system'), Model(id='gpt-5.1', created=1762800673, object='model', owned_by='system'), Model(id='gpt-5.1-codex', created=1762988221, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='mode

In [16]:
model.data

[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-5.1-codex-mini', created=1763007109, object='model', owned_by='system'),
 Model(id='gpt-5.1-chat-latest', created=1762547951, object='model', owned_by='system'),
 Model(id='gpt-5.1-2025-11-13', created=1762800353, object='model', owned_by='system'),
 Model(id='gpt-5.1', created=1762800673, object='model', owned_by='system'),
 Model(id='gpt-5.1-codex', created=1762988221, object='model', owned_by='system'),
 Model(id='davinci-002', created=1692634301, object='model', owned_by='system'),
 Model(id='babbage-002', created=1692634615, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_

In [17]:
print(model.data[0].id)

gpt-4-0613


In [18]:
for m in model.data:
    print(m.id)

gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-5.1-codex-mini
gpt-5.1-chat-latest
gpt-5.1-2025-11-13
gpt-5.1
gpt-5.1-codex
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-realtime-preview-2024-10-01
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini

In [19]:
for m in model.data:
    if any(k in m.id.lower() for k in ["gpt-4o", "gpt-4.1", "gpt-5", "gpt-oss"]):
        print(m.id)

gpt-5.1-codex-mini
gpt-5.1-chat-latest
gpt-5.1-2025-11-13
gpt-5.1
gpt-5.1-codex
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-realtime-preview-2024-10-01
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-4o-realtime-preview-2025-06-03
gpt-4o-audio-preview-2025-06-03
gpt-4o-transcribe-diarize
gpt-5-chat-latest
gpt-5-2025-08-07
gpt-5
gpt-5-mini-2025-08-07
gpt-5-mini
gpt-5-nano-2025-08-07
gpt-5-nano
gpt-5-codex
gpt-5

In [39]:
import re
gpt_pattern = ["gpt-4o", "gpt-4.1", "gpt-5", "gpt-oss"]

for m in model.data:
    model_id = m.id.lower()

    date_pattern = re.compile(r"\d{4}-\d{2}-\d{2}")

    include = any(k in model_id for k in ["gpt-4o", "gpt-4.1", "gpt-5", "gpt-oss"])
    exclude_type = all(k not in m.id.lower() for k in ["image", "realtime", "tts", "audio", "transcribe", "codex", "search", "preview"])
    exclude_mini = all(k not in m.id.lower() for k in ["gpt-4.1-mini", "gpt-4.1-nano", 'gpt-4o-mini'])
    latest_or_date = ("latest" in model_id) or bool(date_pattern.search(model_id))
    if include and exclude_type and exclude_mini:
        print(m.id)

gpt-5.1-chat-latest
gpt-5.1-2025-11-13
gpt-5.1
gpt-4o
gpt-4o-2024-05-13
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-2024-11-20
gpt-4.1-2025-04-14
gpt-4.1
gpt-5-chat-latest
gpt-5-2025-08-07
gpt-5
gpt-5-mini-2025-08-07
gpt-5-mini
gpt-5-nano-2025-08-07
gpt-5-nano
gpt-5-pro-2025-10-06
gpt-5-pro


In [38]:
import re
gpt_pattern = ["gpt-4o", "gpt-4.1", "gpt-5", "gpt-oss"]

for m in model.data:
    model_id = m.id.lower()

    date_pattern = re.compile(r"\d{4}-\d{2}-\d{2}")

    include = any(k in model_id for k in ["gpt-4o", "gpt-4.1", "gpt-5", "gpt-oss"])
    exclude_type = all(k not in m.id.lower() for k in ["image", "realtime", "tts", "audio", "transcribe", "codex", "search", "preview"])
    exclude_mini = all(k not in m.id.lower() for k in ["gpt-4.1-mini", "gpt-4.1-nano", 'gpt-4o-mini'])
    latest_or_date = ("latest" in model_id) or bool(date_pattern.search(model_id))
    if include and exclude_type and exclude_mini and latest_or_date:
        print(m.id)

gpt-5.1-chat-latest
gpt-5.1-2025-11-13
gpt-4o-2024-05-13
gpt-4o-2024-08-06
chatgpt-4o-latest
gpt-4o-2024-11-20
gpt-4.1-2025-04-14
gpt-5-chat-latest
gpt-5-2025-08-07
gpt-5-mini-2025-08-07
gpt-5-nano-2025-08-07
gpt-5-pro-2025-10-06


In [ ]:
for m in model.data:
    if any(k in m.id.lower() for k in ["gpt-4o", "gpt-4.1", "gpt-5", "gpt-oss"]) and all(k not in m.id.lower() for k in ["image", "realtime", "tts", "audio", "transcribe", "codex", "search", "preview"]) and all(k not in m.id.lower() for k in ["gpt-4.1-mini", "gpt-4.1-nano", 'gpt-4o-mini']):
        print(m.id)

In [ ]:
for m in model.data:
    if any(k in m.id.lower() for k in ["dall-e-3", "image"]):
        print(m.id)

In [ ]:
for m in model.data:
    if "tts" in m.id.lower():
        print(m.id)

In [ ]:
for m in model.data:
    if any(k in m.id.lower() for k in ["whisper", "transcribe"]):
        print(m.id)

In [ ]:
for m in model.data:
    if "text-embedding-3" in m.id.lower():
        print(m.id)

In [ ]:
for m in model.data:
    if any(k in m.id.lower() for k in ["audio", "realtime"]):
        print(m.id)

In [ ]:
for m in model.data:
    if "sora" in m.id.lower():
        print(m.id)

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [40]:
from anthropic import Anthropic

In [41]:
client = Anthropic()

In [42]:
client.models.list()

SyncPage[ModelInfo](data=[ModelInfo(id='claude-opus-4-5-20251101', created_at=datetime.datetime(2025, 11, 24, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4.5', type='model'), ModelInfo(id='claude-haiku-4-5-20251001', created_at=datetime.datetime(2025, 10, 15, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Haiku 4.5', type='model'), ModelInfo(id='claude-sonnet-4-5-20250929', created_at=datetime.datetime(2025, 9, 29, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 4.5', type='model'), ModelInfo(id='claude-opus-4-1-20250805', created_at=datetime.datetime(2025, 8, 5, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4.1', type='model'), ModelInfo(id='claude-opus-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4', type='model'), ModelInfo(id='claude-sonnet-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Cl

In [43]:
client.models.list().data

[ModelInfo(id='claude-opus-4-5-20251101', created_at=datetime.datetime(2025, 11, 24, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4.5', type='model'),
 ModelInfo(id='claude-haiku-4-5-20251001', created_at=datetime.datetime(2025, 10, 15, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Haiku 4.5', type='model'),
 ModelInfo(id='claude-sonnet-4-5-20250929', created_at=datetime.datetime(2025, 9, 29, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 4.5', type='model'),
 ModelInfo(id='claude-opus-4-1-20250805', created_at=datetime.datetime(2025, 8, 5, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4.1', type='model'),
 ModelInfo(id='claude-opus-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Opus 4', type='model'),
 ModelInfo(id='claude-sonnet-4-20250514', created_at=datetime.datetime(2025, 5, 22, 0, 0, tzinfo=datetime.timezone.utc), display_name='Claude Sonnet 4', type

In [ ]:
for m in client.models.list().data:
    print(m.id)

In [45]:
for m in client.models.list().data:
    exclude = "claude-3" not in m.id.lower()
    if exclude:
        print(m.id)

claude-opus-4-5-20251101
claude-haiku-4-5-20251001
claude-sonnet-4-5-20250929
claude-opus-4-1-20250805
claude-opus-4-20250514
claude-sonnet-4-20250514


In [82]:
import os
from google import genai

In [97]:
client = genai.Client(http_options={'api_version': 'v1beta'})

In [98]:
client.models.list().page

[Model(
   description='Obtain a distributed representation of a text.',
   display_name='Embedding Gecko',
   input_token_limit=1024,
   name='models/embedding-gecko-001',
   output_token_limit=1,
   supported_actions=[
     'embedText',
     'countTextTokens',
   ],
   tuned_model_info=TunedModelInfo(),
   version='001'
 ),
 Model(
   description='Preview release (Nov 25th, 2025) of Gemini 2.5 Pro with VTEA and DA CSI',
   display_name='Gemini 2.5 Pro Preview with VTEA and DA CSI',
   input_token_limit=1048576,
   max_temperature=2.0,
   name='models/gemini-2.5-pro-vtea-da-csi',
   output_token_limit=65536,
   supported_actions=[
     'generateContent',
     'countTokens',
     'createCachedContent',
     'batchGenerateContent',
   ],
   temperature=1.0,
   thinking=True,
   top_k=64,
   top_p=0.95,
   tuned_model_info=TunedModelInfo(),
   version='2.5-pro-vtea-da-csi'
 ),
 Model(
   description='Gemini 2.5 Pro Preview 03-25',
   display_name='Gemini 2.5 Pro Preview 03-25',
   input_

In [99]:
client.models.list().page[0].display_name

'Embedding Gecko'

In [101]:
response = client.models.list()

In [102]:
response.page

[Model(
   description='Obtain a distributed representation of a text.',
   display_name='Embedding Gecko',
   input_token_limit=1024,
   name='models/embedding-gecko-001',
   output_token_limit=1,
   supported_actions=[
     'embedText',
     'countTextTokens',
   ],
   tuned_model_info=TunedModelInfo(),
   version='001'
 ),
 Model(
   description='Preview release (Nov 25th, 2025) of Gemini 2.5 Pro with VTEA and DA CSI',
   display_name='Gemini 2.5 Pro Preview with VTEA and DA CSI',
   input_token_limit=1048576,
   max_temperature=2.0,
   name='models/gemini-2.5-pro-vtea-da-csi',
   output_token_limit=65536,
   supported_actions=[
     'generateContent',
     'countTokens',
     'createCachedContent',
     'batchGenerateContent',
   ],
   temperature=1.0,
   thinking=True,
   top_k=64,
   top_p=0.95,
   tuned_model_info=TunedModelInfo(),
   version='2.5-pro-vtea-da-csi'
 ),
 Model(
   description='Gemini 2.5 Pro Preview 03-25',
   display_name='Gemini 2.5 Pro Preview 03-25',
   input_

In [103]:
models = []
for m in response.page:
    models.append(m.name)

In [104]:
llm = []
for m in response.page:
    if "generateContent" in m.supported_actions:
        llm.append(m.name)

In [105]:
llm

['models/gemini-2.5-pro-vtea-da-csi',
 'models/gemini-2.5-pro-preview-03-25',
 'models/gemini-2.5-flash',
 'models/gemini-2.5-pro-preview-05-06',
 'models/gemini-2.5-pro-preview-06-05',
 'models/gemini-2.5-pro',
 'models/gemini-2.0-flash-exp',
 'models/gemini-2.0-flash',
 'models/gemini-2.0-flash-001',
 'models/gemini-2.0-flash-exp-image-generation',
 'models/gemini-2.0-flash-lite-001',
 'models/gemini-2.0-flash-lite',
 'models/gemini-2.0-flash-lite-preview-02-05',
 'models/gemini-2.0-flash-lite-preview',
 'models/gemini-2.0-pro-exp',
 'models/gemini-2.0-pro-exp-02-05',
 'models/gemini-exp-1206',
 'models/gemini-2.0-flash-thinking-exp-01-21',
 'models/gemini-2.0-flash-thinking-exp',
 'models/gemini-2.0-flash-thinking-exp-1219',
 'models/gemini-2.5-flash-preview-tts',
 'models/gemini-2.5-pro-preview-tts',
 'models/learnlm-2.0-flash-experimental',
 'models/gemma-3-1b-it',
 'models/gemma-3-4b-it',
 'models/gemma-3-12b-it',
 'models/gemma-3-27b-it',
 'models/gemma-3n-e4b-it',
 'models/gemm

In [106]:
models

['models/embedding-gecko-001',
 'models/gemini-2.5-pro-vtea-da-csi',
 'models/gemini-2.5-pro-preview-03-25',
 'models/gemini-2.5-flash',
 'models/gemini-2.5-pro-preview-05-06',
 'models/gemini-2.5-pro-preview-06-05',
 'models/gemini-2.5-pro',
 'models/gemini-2.0-flash-exp',
 'models/gemini-2.0-flash',
 'models/gemini-2.0-flash-001',
 'models/gemini-2.0-flash-exp-image-generation',
 'models/gemini-2.0-flash-lite-001',
 'models/gemini-2.0-flash-lite',
 'models/gemini-2.0-flash-lite-preview-02-05',
 'models/gemini-2.0-flash-lite-preview',
 'models/gemini-2.0-pro-exp',
 'models/gemini-2.0-pro-exp-02-05',
 'models/gemini-exp-1206',
 'models/gemini-2.0-flash-thinking-exp-01-21',
 'models/gemini-2.0-flash-thinking-exp',
 'models/gemini-2.0-flash-thinking-exp-1219',
 'models/gemini-2.5-flash-preview-tts',
 'models/gemini-2.5-pro-preview-tts',
 'models/learnlm-2.0-flash-experimental',
 'models/gemma-3-1b-it',
 'models/gemma-3-4b-it',
 'models/gemma-3-12b-it',
 'models/gemma-3-27b-it',
 'models/

In [118]:
llm = []
for m in response.page:
    include = any(k in m.name.lower() for k in ["gemini", "gemma"])
    exclude_type = all(k not in m.name.lower() for k in ["embedding", "tts", "exp"])
    exclude_model = all(k not in m.name.lower() for k in ["gemini-2.0"])

    preview_check = ("preview" not in m.name.lower()) or any(k in m.name.lower() for k in ["gemini-3-pro-preview", "gemini-3-pro-image-preview"])
    if "generateContent" in m.supported_actions and include and exclude_type and exclude_model and preview_check:
        llm.append(m.name.split('/')[-1])

llm

['gemini-2.5-pro-vtea-da-csi',
 'gemini-2.5-flash',
 'gemini-2.5-pro',
 'gemma-3-1b-it',
 'gemma-3-4b-it',
 'gemma-3-12b-it',
 'gemma-3-27b-it',
 'gemma-3n-e4b-it',
 'gemma-3n-e2b-it',
 'gemini-flash-latest',
 'gemini-flash-lite-latest',
 'gemini-pro-latest',
 'gemini-2.5-flash-lite',
 'gemini-2.5-flash-image',
 'gemini-3-pro-preview',
 'gemini-3-pro-image-preview']

In [80]:
image_model = []
for m in response.page:
    if "imagen" in m.name.lower():
        image_model.append(m.name)

image_model

['models/imagen-4.0-generate-preview-06-06']

In [17]:
import os

In [18]:
from dotenv import get_key

In [19]:
import perplexity
from perplexity import Perplexity

In [20]:
client = Perplexity(api_key=get_key(dotenv_path=".env", key_to_get="PERPLEXITY_API_KEY"))

In [21]:
try:
    test = client.search.create(query="test")
except perplexity.AuthenticationError:
    print("Invalid API key. Please check your PERPLEXITY_API_KEY environment variable.")


In [ ]:
import xai_sdk

In [ ]:
client = xai_sdk.Client()

In [ ]:
models = client.models.list_language_models()

In [ ]:
import lmstudio as lms

In [ ]:
llm = lms.list_downloaded_models("llm")

In [ ]:
for m in llm:
    print(m.model_key)

In [ ]:
import ollama

In [ ]:
models = ollama.list()

In [ ]:
for m in models.models:
    print(m.model)

In [1]:
import requests

In [2]:
server_address="127.0.0.1:8000"

In [3]:
model_route = f"http://{server_address}/models/checkpoints"

In [4]:
models = requests.get(model_route)

In [6]:
models.content[1:-1]

b'"ilxl/Illustrious-XL-v1.0.safetensors", "sd15/AOM3A1B_orangemixs.safetensors", "sd15/Anything-ink.safetensors", "sd15/AnythingXL_v50.safetensors", "sd15/Counterfeit-V3.0_fix_fp16.safetensors", "sd15/anythingV3_fp16.safetensors", "sd15/sd-v1-5-inpainting.ckpt", "sd15/v1-5-pruned-emaonly.safetensors", "sd35_large/emi3.safetensors", "sdxl/AnythingXL_xl.safetensors", "sdxl/AnythingXL_xlHyper8stepsCFG1.safetensors", "sdxl/animagine-xl-3.1.safetensors", "sdxl/animagine-xl-4.0-opt.safetensors", "sdxl/fp8/animagine-xl-4.0-opt-fp8.safetensors", "sdxl/sd_xl_base_1.0.safetensors", "sdxl/sd_xl_refiner_1.0.safetensors"'

In [5]:
checkpoints_byte = models.content[1:-1]

In [7]:
checkpoints = models.content[1:-1].decode()

In [8]:
checkpoints

'"ilxl/Illustrious-XL-v1.0.safetensors", "sd15/AOM3A1B_orangemixs.safetensors", "sd15/Anything-ink.safetensors", "sd15/AnythingXL_v50.safetensors", "sd15/Counterfeit-V3.0_fix_fp16.safetensors", "sd15/anythingV3_fp16.safetensors", "sd15/sd-v1-5-inpainting.ckpt", "sd15/v1-5-pruned-emaonly.safetensors", "sd35_large/emi3.safetensors", "sdxl/AnythingXL_xl.safetensors", "sdxl/AnythingXL_xlHyper8stepsCFG1.safetensors", "sdxl/animagine-xl-3.1.safetensors", "sdxl/animagine-xl-4.0-opt.safetensors", "sdxl/fp8/animagine-xl-4.0-opt-fp8.safetensors", "sdxl/sd_xl_base_1.0.safetensors", "sdxl/sd_xl_refiner_1.0.safetensors"'

In [ ]:
checkpoint_list = [x.decode('utf-8') for x in checkpoints_byte.split(b', ') if x]

In [ ]:
print(checkpoint_list)

In [ ]:
print(checkpoints)
print(type(checkpoints))

In [9]:
checkpoint_list = [x for x in checkpoints.replace("\"", "").strip().split(', ') if x]

In [10]:
checkpoint_list

['ilxl/Illustrious-XL-v1.0.safetensors',
 'sd15/AOM3A1B_orangemixs.safetensors',
 'sd15/Anything-ink.safetensors',
 'sd15/AnythingXL_v50.safetensors',
 'sd15/Counterfeit-V3.0_fix_fp16.safetensors',
 'sd15/anythingV3_fp16.safetensors',
 'sd15/sd-v1-5-inpainting.ckpt',
 'sd15/v1-5-pruned-emaonly.safetensors',
 'sd35_large/emi3.safetensors',
 'sdxl/AnythingXL_xl.safetensors',
 'sdxl/AnythingXL_xlHyper8stepsCFG1.safetensors',
 'sdxl/animagine-xl-3.1.safetensors',
 'sdxl/animagine-xl-4.0-opt.safetensors',
 'sdxl/fp8/animagine-xl-4.0-opt-fp8.safetensors',
 'sdxl/sd_xl_base_1.0.safetensors',
 'sdxl/sd_xl_refiner_1.0.safetensors']

In [ ]:
for m in checkpoint_list:
    print(m)

In [3]:
import ComfyUI

In [ ]:
import transformers

In [3]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import mimetypes
import os
from google import genai
from google.genai import types


def save_binary_file(file_name, data):
    f = open(file_name, "wb")
    f.write(data)
    f.close()
    print(f"File saved to to: {file_name}")


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-3-pro-image-preview"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""INSERT_INPUT_HERE"""),
            ],
        ),
    ]
    tools = [
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_modalities=[
            "IMAGE",
            "TEXT",
        ],
        image_config=types.ImageConfig(
            aspect_ratio="16:9",
            image_size="4K",
        ),
        tools=tools,
    )

    file_index = 0
    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if (
            chunk.candidates is None
            or chunk.candidates[0].content is None
            or chunk.candidates[0].content.parts is None
        ):
            continue
        if chunk.candidates[0].content.parts[0].inline_data and chunk.candidates[0].content.parts[0].inline_data.data:
            file_name = f"ENTER_FILE_NAME_{file_index}"
            file_index += 1
            inline_data = chunk.candidates[0].content.parts[0].inline_data
            data_buffer = inline_data.data
            file_extension = mimetypes.guess_extension(inline_data.mime_type)
            save_binary_file(f"{file_name}{file_extension}", data_buffer)
        else:
            print(chunk.text)

if __name__ == "__main__":
    generate()


Here are a few options ranging from direct to slightly more conversational. Choose the one that best fits your relationship with your boss.

**Option 1: Clear
 and direct (Best for most situations)**

**Subject:** WFH Request for Tomorrow, [insert date here] – [Your Name]

Hi [Boss's
 Name],

I would like to request to work from home tomorrow, [insert day, e.g., Tuesday].

I need to be present at my apartment to let
 in a repairman for some necessary maintenance during business hours.

Please be assured that I will be fully connected with stable internet and reachable via email and chat as usual. I
 do not anticipate any disruption to my workflow.

Please let me know if this works for you.

Best regards,

[Your Name]

***

**Option 
2: Slightly softer tone (If you have a casual rapport)**

**Subject:** Working from home tomorrow - [Your Name]

Hi [Boss's
 Name],

Would it be okay if I logged in from home tomorrow, [insert day]?

I have a repairman scheduled to come by my place during t